In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import nibabel as nib
import glob
import os
from pyhere import here
import sys

from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_stat_map, view_img, plot_glass_brain
from nilearn.image import concat_imgs, smooth_img, mean_img, math_img, resample_to_img
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.glm.second_level import non_parametric_inference

In [8]:
sys.path.append(str(here('01_level1')))
from level1_utils import make_level1_design_matrix, make_contrasts
from utils import get_from_sidecar, get_model_regs

## How can l2 z-maps look very constrained but l3 randomise with massive swaths of activation?

- How l3 t values be larger than the largest t values in the individual l2 mean maps?
    - L2 mean maps are generated by `nilearn.glm.second_level.SecondLevelModel.compute_contrast` with argument `output_type="z_score"`
    - L3 t values are computed using FSL's `randomise`  

Previous hypotheses:

- Is it because `rabndomise` uses enhanced TFCE statistics instaed of raw T stats?  
    - No. When I tried voxelwise setting `tfce=False` and checked the min and max values in `randomise_results.outputs.tstat_files` they were the same as the outputs using tfce.  

- Is it because of smoothing?
    - No. Smoothing l2 images made values only smaller.

Current hypothesis:

- Statistic mismatch between levels of analysis.
    - FSL's `randomise` expects `cope` images. These should be beta weights. What `output_type` do these correspond to in `nilearn` terminology?
    - What is the correct input type for l2 and l3 analyses in `nilearn`?
    - What does a max T distribution look like in `nilearn`s permutation testing framework?

## `compute_contrast` output types

Based on the [`Contrast` class](https://github.com/nilearn/nilearn/blob/2fd6665664f27dfbf1f10aeeba660adc9b560361/nilearn/glm/contrasts.py#L143)
 in `nilearn.glm.contrasts`

- `effect_size`: parameter estimate from the GLM defined as [`effect = np.dot(matrix, self.theta)`](https://github.com/nilearn/nilearn/blob/2fd6665664f27dfbf1f10aeeba660adc9b560361/nilearn/glm/model.py#L213)
- `effect_variance`: variance of the parameter estimate from the GLM [`sd = np.sqrt(self.vcov(matrix=matrix, dispersion=dispersion))`](https://github.com/nilearn/nilearn/blob/2fd6665664f27dfbf1f10aeeba660adc9b560361/nilearn/glm/model.py#L213)
- `stat`: decision statistic[`stat = (self.effect - baseline) / np.sqrt(self.variance)`](https://github.com/nilearn/nilearn/blob/2fd6665664f27dfbf1f10aeeba660adc9b560361/nilearn/glm/contrasts.py#L223) comparing the parameter estimate to the null hypothesis (i.e. is it different than 0.)
- `p_value`: p value for the decision statistic [`p_values = scipy.stats.t.sf(self.stat_, self.dof`](https://github.com/nilearn/nilearn/blob/2fd6665664f27dfbf1f10aeeba660adc9b560361/nilearn/glm/contrasts.py#L275)
- `z_score`: z scores for the given p values. Almost identical to `stat`

- Run level 1 analysis on single subject single run
- Run `compute_contrast` for each `output_type`
- Pick a single voxel
- Run GLM on it and compare the value for this voxel in each contrast map to the GLM summary

In [3]:
subnum = '01'
runnum = '1'
mnum = 'model3'

data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

noise_model='ar1'
hrf_model='spm'
drift_model='cosine'
smoothing_fwhm=5

In [4]:
design_matrix= make_level1_design_matrix(subnum, runnum, mnum, data_path, behavior_path, regress_rt = 0)

A 'modulation' column was found in the given events data and is used.


In [5]:
run_events = os.path.join(data_path, 'sub-%s/func/sub-%s_task-bundles_run-%s_events.tsv'%(subnum, subnum, runnum))

In [6]:
#fmri_img: path to preproc_bold that the model will be fit on
fmri_img = os.path.join(data_path,"derivatives/fmriprep/sub-%s/func/sub-%s_task-bundles_run-%s_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz"%(subnum, subnum, runnum))
os.path.isfile(fmri_img)

True

In [9]:
#read in preproc_bold for that run
cur_img_tr = get_from_sidecar(subnum, runnum, 'RepetitionTime', data_path)

#read in events.tsv for that run
cur_events = pd.read_csv(run_events, sep = '\t')

mask_img = nib.load(os.path.join(data_path,'derivatives/fmriprep/sub-%s/func/sub-%s_task-bundles_run-%s_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz'%(subnum, subnum, runnum)))

In [12]:
fmri_glm = FirstLevelModel(t_r=cur_img_tr,
                       noise_model=noise_model,
                       standardize=False,
                       hrf_model=hrf_model,
                       drift_model=drift_model,
                       smoothing_fwhm=smoothing_fwhm,
                       mask_img=mask_img)

In [13]:
fmri_glm = fmri_glm.fit(fmri_img, design_matrices = design_matrix)

In [14]:
contrasts = make_contrasts(design_matrix, mnum)

In [17]:
contrast_val = contrasts['reward_par']

In [18]:
z_map = fmri_glm.compute_contrast(contrast_val, output_type='z_score')
stat_map = fmri_glm.compute_contrast(contrast_val, output_type='stat')
p_map = fmri_glm.compute_contrast(contrast_val, output_type='p_value')
effect_size_map = fmri_glm.compute_contrast(contrast_val, output_type='effect_size')
# effect_variance_map = fmri_glm.compute_contrast(contrast_val, output_type='effect_variance')

In [36]:
p_map_data = p_map.get_fdata()
oneminp_data = 1 - p_map_data
oneminp_map = nib.Nifti1Image(oneminp_data.astype(np.float64), p_map.affine)

In [76]:
# Pick a voxel, the one with the max statistic
np.unravel_index(np.argmax(stat_map.get_fdata()), stat_map.get_fdata().shape)

(64, 83, 32)

## Input type for level2

## Run level3 analysis using nilearn instead of randomise

In [ ]:
# 1's for all runs to get average for subject per contrast
design_matrix = pd.DataFrame([1] * len(l2_imgs), columns=['intercept'])
model = SecondLevelModel(smoothing_fwhm=5.0)

model = model.fit(l2_imgs, design_matrix=design_matrix)

# z_map = model.compute_contrast(output_type='z_score')
z_map = model.compute_contrast(output_type='stat')


Raw map thresholded at an arbitrary large value. What nilearn reports for `output_type = "stat"` is comparable to randomise outputs.

What statistic is reported when `output_type = "stat"`? 

Could this be why things look weird when using randomise? I'm feeding in `z_score` maps insteaf og `stat` maps? What input does randomise expect? `cope` images, contrast of parameters (beta weights)?




In [ ]:
plot_stat_map(z_map, threshold=3, draw_cross=False)


In [ ]:
neg_log_pvals_permuted_ols_unmasked = non_parametric_inference(l2_imgs,
                             design_matrix=design_matrix,
                             model_intercept=True, n_perm=5000,
                             two_sided_test=False,
                             smoothing_fwhm=5.0, n_jobs=1)

In [ ]:
plot_stat_map(
    neg_log_pvals_permuted_ols_unmasked, colorbar=True, 
    threshold=1, draw_cross=False, cut_coords = (-6, -95, 14))
# The neg-log p-values obtained with non parametric testing are capped at 3
# when the number of permutations is 1e3.

What is the max T distribution that lead to the map above? Should be able to get it with the `nilearn.mass_univariate.permuted_ols` function.

### Compared to the randomise output

In [ ]:
plot_stat_map(os.path.join(l3_path, 'rand_tfce_tstat1_pos_overall-mean_reward_par_model3_reg-rt0.nii.gz'), threshold = 4.6, draw_cross=False)